In [2]:
!pip install langchain

In [3]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 25.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


In [5]:
from langchain_groq import ChatGroq

In [6]:
llm = ChatGroq(
    temperature = 0,
    groq_api_key='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',
   model="llama-3.1-70b-versatile"
)


In [9]:
%pip install -qU langchain_community beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-39275?from=job%20search%20funnel")
page_data = loader.load().pop().page_content

In [12]:
page_data

'Solliciteren voor NIKE, Inc. Consumer, Product, Brand & Marketplace Operations (CPBM) Graduate Internship\n\nZoek vacaturesNavigatie overslaanZoek vacaturesNIKE, INC. JOBSFuncties op opdrachtbasisJoin The Talent CommunityLife @ NikeOverzichtArbeidsvoorwaardenMerkenOverzichtJordanConverseTeamsOverzichtAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFaciliteitenFinance & AccountingGovernment & Public AffairsHuman ResourcesData & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSociale en community-impactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocatiesOverzichtNike WHQNike New York HQEHQ: Hilversum, NederlandELC: Laakdal, BelgiumGreater China HQDiversiteit, gelijkheid en inclusieOverzichtMilitairenBeperking en inclusieInclusie van Native AmericansS

After some iterations I got this nice looking prompt that gives me the best result. Then I have used a pipe operation to create a chain.

In [16]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED DATA FROM THE WEBSITE:
    {page_data}
    ### Instruction:
    The scraped data is from the career's page of a website.
    Your Job is to extract the job posting and return them in JSON format containing the following keys: 'role', 'experience', 'skills' and 'description'.
    only return the JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data' :page_data})
print(res.content)


```json
{
    "role": "NIKE, Inc. Consumer, Product, Brand & Marketplace Operations (CPBM) Graduate Internship",
    "experience": "3-5 years work experience",
    "skills": [
        "Proven ability to partner on cross-functional teams supporting large scale change across an organization",
        "Passionate about processes potential for improving planning, execution, and work/life balance",
        "Excellent problem structuring and strategic problem-solving skills",
        "Compelling, high-impact storyteller with the ability to influence at multiple levels throughout the organization",
        "Self-directed and comfortable working in ambiguous environments",
        "Knowledge of product creation, design thinking and merchandising strongly preferred",
        "Strong experience with digital tools and demonstrated digital approach",
        "Passionate about Nike brand, products and consumers"
    ],
    "description": "As a part of the CPBM Operations team, you will work on proj

After some iterations I got this nice looking prompt that gives me the best result. As we can see that just by providing the url for page, we got the role, experience, skills and description.

In [17]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED DATA FROM THE WEBSITE:
    {page_data}
    ### Instruction:
    The scraped data is from the career's page of a website.
    Your Job is to extract the job posting and return them in JSON format containing the following keys: 'role', 'experience', 'skills' and 'description'.
    only return the JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data' :page_data})
type(res.content)

str

As the output type is text. we will now convert the text format into JSON format using the JSONparser. The output type will then be in dict(dictionary) format.

In [19]:

from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

{'role': 'NIKE, Inc. Consumer, Product, Brand & Marketplace Operations (CPBM) Graduate Internship', 'experience': '3-5 years work experience', 'skills': ['Proven ability to partner on cross-functional teams supporting large scale change across an organization', 'Passionate about processes potential for improving planning, execution, and work/life balance', 'Excellent problem structuring and strategic problem-solving skills', 'Compelling, high-impact storyteller with the ability to influence at multiple levels throughout the organization', 'Self-directed and comfortable working in ambiguous environments', 'Knowledge of product creation, design thinking and merchandising strongly preferred', 'Strong experience with digital tools and demonstrated digital approach', 'Passionate about Nike brand, products and consumers'], 'description': 'As a part of the CPBM Operations team, you will work on projects that focus on accountability and consistency of process, tools, results, and communication

In [20]:
type(json_res)

dict

Now we will prepare ChromaDB. I have created a CSV file that has TechStack and related links to the portfolio. The links will be entered in the ChromaDB. So for a given job posting, the created prompt will extract the skills and then match one or multiple technologies in the CSV. Then we will retrieve the portfolio URL to add into the EMAIL.

In [22]:
from google.colab import files


uploaded = files.upload()


Saving my_portfolio.csv to my_portfolio.csv


In [23]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")

In [24]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


Now we will insert the above one by one into ChromaDB.

In [32]:
!pip install chromadb

import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="my_collection")

if not collection.count():
  for _, row in df.iterrows():
    collection.add(documents=row["Techstack"],
                  metadatas={"links": row ["Links"]},
                   ids=[str(uuid.uuid4())])

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 27.6MiB/s]


I am going to test if I am able to get a relevant link for given text.

In [37]:
Links = collection.query(query_texts=["Need experience in python, React"], n_results=2).get('metadatas', [])
Links

[[{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

If we use skills from the past result. We should get relevant portfolio links for all keywords.


In [41]:
Links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
Links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}]]

In [38]:
job = json_res
job['skills']

['Proven ability to partner on cross-functional teams supporting large scale change across an organization',
 'Passionate about processes potential for improving planning, execution, and work/life balance',
 'Excellent problem structuring and strategic problem-solving skills',
 'Compelling, high-impact storyteller with the ability to influence at multiple levels throughout the organization',
 'Self-directed and comfortable working in ambiguous environments',
 'Knowledge of product creation, design thinking and merchandising strongly preferred',
 'Strong experience with digital tools and demonstrated digital approach',
 'Passionate about Nike brand, products and consumers']

Creating a prompt for creating an cold email.

In [42]:
prompt_extract = PromptTemplate.from_template(
     """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Om, a business development executive at TCS. TCS is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of TCS
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase TCS's portfolio: {link_list}
        Remember you are Om, BDE at TCS.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke({"job_description": str(job), "link_list": Links})
print(res.content)


Subject: Unlocking Operational Efficiency for Nike's CPBM Operations Team

Dear Hiring Manager,

I came across the Consumer, Product, Brand & Marketplace Operations (CPBM) Graduate Internship role at Nike, Inc. and was impressed by the organization's commitment to simplifying processes and driving efficiency. As a Business Development Executive at TCS, I believe our expertise in process optimization and digital transformation can help Nike achieve its goals.

At TCS, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in designing and implementing digital solutions that drive business growth and improve operational efficiency.

In particular, our expertise in DevOps, digital transformation, and process optimization can help Nike's CPBM Operations team streamline processes, improve communication, and drive consistency 

The given prompt inludes the job description as well as the portfolio link aalong with a well-defined prompt.